# Edge-Vertex Continuous Collision Detection

Given a point that moves over time, $\vec{p}_0(t)$, and an edge, $\vec{e}(s, t)$, comprised of two points, $\vec{p}_1(t)$ and $\vec{p}_2(t)$, we want to find the time of impact between the point and edge. Let us assume a normalized time step $t=0$ to $t=1$. We known the point positions for $t=0$ and $t=1$. Let us also assume that for all points the trajectory is linear and the velocity is constant for all points. We can write the trajectories as 

\begin{align}
    \vec{p}_i(t) &= [\vec{p}_i(1) - \vec{p}_i(0)] t + \vec{p}_i(0) & \forall i \in \{0, 1, 2\}, t \in [0, 1]\\
    e(s, t) &= (\vec{p}_2(t) - \vec{p}_1(t)) s + \vec{p}_1(t) & \forall s \in [0, 1] 
\end{align}

In [1]:
# Compute a polynomial for the intersection of a point and edge both moving
# through time.
# General idea: http://www.sci.utah.edu/~kpotter/publications/ramsey-2004-RBPI.pdf

import numpy
import sympy
from IPython.display import display, Markdown, Latex
# sympy.init_printing(use_latex='mathjax')

# s is the spatial parameterization for the edge
# t is the temporal parameterization
s, t = sympy.symbols("s t")

In [2]:
dimensions = 2  # Number of dimensions of the points and edge
# For each dimenstion write the trajectory of the point and edge
d = "y" if dimensions == 2 else "z"
# Three points, one free and two for the edge, each with a velocity
p = numpy.reshape(sympy.symbols(f"p_0:3_x:{d}"), (-1, dimensions))
v = numpy.reshape(sympy.symbols(f"v_0:3_x:{d}"), (-1, dimensions))

In [3]:
# Write the point's trajectory as a line in time
pt = p + t * v
# Write the dimensions of the edge
e = (pt[2,:] - pt[1,:]) * s + pt[1,:]

In [4]:
sn = [None] * dimensions
sd = [None] * dimensions
for i in range(0, dimensions):
    # Equate the point and edge in the current dimension
    eq = sympy.Eq(pt[0,i], e[i])
    # Solve for the spatial parameterization along the edge
    s_ = sympy.solve(eq, s)[0]
    display(Latex(f"$s_{i} = {s_}$"))
    # Split the spatial parameterization into a numerator and denominator
    # NB: The denominator can be zero if the edge is degenerate
    sn[i], sd[i] = sympy.fraction(s_)

# Equate the different dimensions of the spatial parameterization
if dimensions == 2:
    eq1 = sympy.Eq(sn[0] * sd[1], sn[1] * sd[0])
elif dimensions == 3:
    eq1 = sympy.Eq(sd[2] * (sn[0] * sd[1] - sn[1] * sd[0]), sn[0])
else:
    raise Exception("Invalid dimension option")

# Write the equations as a polynomial in t
t_poly = sympy.polys.polytools.poly_from_expr(eq1, t)

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

In [5]:
display(Latex("$a_2t^2 + a_1t + a_0 = 0$"))
coeffs = t_poly[0].all_coeffs()
for i, coeff in enumerate(coeffs):
    display(Latex(f"$a_{dimensions - i} = {sympy.factor(coeff)}$"))

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

## Equation for $t$ in 2D:

\begin{align*}
&[v_{0x}(v_{2y}-v_{1y}) + v_{0y}(v_{1x} - v_{2x}) - v_{1x}v_{2y} + v_{1y}v_{2x}] \cdot t^2 \\
+& [p_{0x}(v_{2y}-v_{1y}) + p_{0y}(v_{1x} - v_{2x}) + p_{1x}(v_{0y} - v_{2y}) + p_{1y}(v_{2x}-v_{0x}) + p_{2x}(v_{1y}-v_{0y}) + p_{2y}(v_{0x} - v_{1x})] \cdot t \\
+& p_{0x}(p_{2y} - p_{1y}) + p_{0y}(p_{1x} - p_{2x}) - p_{1x}p_{2y} + p_{1y}p_{2x} = 0 \\
\end{align*}